## Test and Error

As we continue developing our package, many errors might arised without noticing. To prevent this, the best way is to write a test suite.

**Test suite:** 
- It is a set of tests that should be run automatically to check every functionality of our package every time we update its distribution
- The test suite is stored in the tests folder of your package root directory.
- All the files in the tests folder are called test_name_of_test.py where name_of_test should be replaced by the name of the module we are testing.

For instance, in the company package, we can create the following test files:

In [ ]:
tests/
├── test_base_company.py
├── test_cli.py
└── test_medical.py

A test file looks contains a set of test functions that look like this. These functions are based on the **assert statement**. The assert statement is used to check if a condition is true. If the condition is false, an AssertionError is raised.

In [ ]:
# we should have at least one test function for each method of each class in the module
def test_class_init():
    instance = class_name(pararm1="MediCorp", pararm2="Cardiology", pararm3=True)
    assert instance.pararm1 == "MediCorp"
    assert instance.pararm2 == "Cardiology"
    assert instance.pararm3 is True

### Pytest

- **pytest** is a library we used for testing. 
- It can run the whole test suites
- pytest provides fixtures (some sort of code that pytest inject when running test suites)
- below we use **capsys** and **monkeypatch** two fixtures for testing CLI command.

In [ ]:
def test_get_stock_price_difference(capsys, monkeypatch):
    # capsys: pytest fixture that captures system output
    # monkeypatch: pytest fixture used to mock command-line arguments (that's why we have "sys.argv" below, monkeypatch can also mock env variables etc.)


    # mock command-line arguments with a known ticker and date range
    monkeypatch.setattr("sys.argv", [
        "cli.py", "get_stock_price_difference",
        "--ticker", "AAPL",
        "--interval", "1y",
        "--stop_date", "2023-12-31"
    ])

    # run the CLI main function
    main() #rememberthat this starts the CLI we defined in cli.py

    # capture output
    captured = capsys.readouterr()
    # capsys.readouterr() return an object with two attributes: .out for what system print, .err for system error

    # test the numeric value directly by extracting it from the output
    price_diff = float(captured.out.split(": ")[1].strip())
    assert abs(price_diff - 18.717864990234) < 1e-4

pytest contains a nice feature to compare floating point numbers with a certain precision, which is **pytest.approx**

In [ ]:
import pytest
def test_get_stock_price_difference(capsys, monkeypatch):
    # capsys: pytest fixture that captures system output
    # monkeypatch: pytest fixture used to mock command-line arguments (that's why we have "sys.argv" below, monkeypatch can also mock env variables etc.)


    # mock command-line arguments with a known ticker and date range
    monkeypatch.setattr("sys.argv", [
        "cli.py", "get_stock_price_difference",
        "--ticker", "AAPL",
        "--interval", "1y",
        "--stop_date", "2023-12-31"
    ])

    # run the CLI main function
    main() #rememberthat this starts the CLI we defined in cli.py

    # capture output
    captured = capsys.readouterr()
    # capsys.readouterr() return an object with two attributes: .out for what system print, .err for system error

    # test the numeric value directly by extracting it from the output
    price_diff = float(captured.out.split(": ")[1].strip())
    assert price_diff == pytest.approx(18.717864990234, rel=1e-4)

To test all entries in an array as input we can also use **np.testing.assert_allclose**

Finally, we can use pytest to run the test suites. To do so, we go to the root directory of the package and run: (instaed of running each test function ourselves)

In [ ]:
pytest -s tests/*
# run all the tests in the tests folder
# -s option: show the output of any print statements in test files on the terminal

If we want to run a single test, we can use the following command:

In [ ]:
pytest tests/test_<name of test>.py

When a test runs well we would see something like this:

In [ ]:
================================================== test session starts ==================================================
platform darwin -- Python 3.9.13, pytest-7.2.0, pluggy-1.0.0
rootdir: /Users/boris/MPhil/company_package
plugins: cov-4.1.0, anyio-3.6.2
collecting ... Company package version: 0.0.0b1.dev8+g5c0d18a.d20241030
collected 8 items

tests/test_base_company.py ....
tests/test_cli.py ..
tests/test_medical.py ..

=================================================== 8 passed in 0.85s ===================================================

When a test fails we would see something like this, it shows which function fails

In [ ]:
================================================== test session starts ==================================================
platform darwin -- Python 3.9.13, pytest-7.2.0, pluggy-1.0.0
rootdir: /Users/boris/MPhil/company_package
plugins: cov-4.1.0, anyio-3.6.2
collecting ... Company package version: 0.0.0b1.dev8+g5c0d18a.d20241030
collected 8 items

tests/test_base_company.py ....
tests/test_cli.py .F
tests/test_medical.py ..

======================================================= FAILURES ========================================================
____________________________________________ test_get_stock_price_difference ____________________________________________

capsys = <_pytest.capture.CaptureFixture object at 0x134d294c0>
monkeypatch = <_pytest.monkeypatch.MonkeyPatch object at 0x134d297c0>

    def test_get_stock_price_difference(capsys, monkeypatch):
        # Mock command-line arguments with a known ticker and date range
        monkeypatch.setattr("sys.argv", [
            "cli.py", "get_stock_price_difference",
            "--ticker", "AAPL",
            "--interval", "1y",
            "--stop_date", "2023-12-31"
        ])

        # Run the CLI main function
        main()

        # Capture output
        captured = capsys.readouterr()

        # # Test the numeric value directly by extracting it from the output
        price_diff = float(captured.out.split(": ")[1].strip())
        # assert abs(price_diff - 18.717864990234) < 1e-4


        # Test using pytest.approx for better floating point comparison
>       assert price_diff == pytest.approx(19.717864990234, rel=1e-4)
E       assert 18.717864990234375 == 19.717864990234 ± 2.0e-03
E         comparison failed
E         Obtained: 18.717864990234375
E         Expected: 19.717864990234 ± 2.0e-03

tests/test_cli.py:42: AssertionError
================================================ short test summary info ================================================
FAILED tests/test_cli.py::test_get_stock_price_difference - assert 18.717864990234375 == 19.717864990234 ± 2.0e-03
============================================== 1 failed, 7 passed in 0.89s ==============================================

### Types of errors 

- **ZeroDivisionError:** Raised when attempting to divide by zero.
- **TypeError:** Raised when an operation or function is applied to an object of inappropriate type. For example, trying to add a string to an integer
- **ValueError:** Raised when a function receives an argument of the correct type but inappropriate value. This could happen, for instance, when trying to convert a non-numeric string to an integer.
- **IndexError:** Raised when an index is out of the range of a list, tuple, or other indexable collections.
- **KeyError:** Raised when trying to access a dictionary with a key that doesn’t exist. 
- **AttributeError:** Raised when an invalid attribute is referenced, typically due to accessing an attribute or method that doesn’t exist in an object.
- **FileNotFoundError:** Raised when trying to open a file that does not exist. 
- **OverflowError:** Raised when a numerical calculation exceeds the maximum limit for a numeric type. 
- **AssertionError:** Raised when an assert statement fails. Useful in testing when specific conditions should be met.
- **RuntimeError:** A generic error raised when an error occurs that doesn’t fall into other categories.

**Exception handling:** the exception (error) types above help us to do exception handling; that is, when specific type of error exoist, we can let the program continue (not crash) while sending a message to the user to inform what is wrong.

An example of exception handling:

In [3]:
def divide(a, b):
    try:
        result = a / b
    except ZeroDivisionError:
        print("Error: Cannot divide by zero!")
        return None
    else:
        print("Division successful!")
        return result
    finally:
        print("Execution complete.")

# Example usage
print(divide(10, 2))  # Should print "Division successful!" and the result 5.0
print(divide(10, 0))  # Should print "Error: Cannot divide by zero!" and return None


Division successful!
Execution complete.
5.0
Error: Cannot divide by zero!
Execution complete.
None
